In [1]:
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
from PIL import Image

import torch
import yaml

import os
import random
from ultralytics.utils.plotting import plot_labels
import pandas as pd
import sys

module_path = os.path.abspath(os.path.join('..'))
print(module_path)
module_path = module_path+'/data_preprocessing'
print(module_path)

if module_path not in sys.path:
    sys.path.append(module_path)

import visualization_utils as visutils

Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA RTX A5000, 24248MiB)
Setup complete ✅ (32 CPUs, 251.6 GB RAM, 0.2/125.8 GB disk)


/vast/palmer/home.grace/eec42/BirdDetector/src
/vast/palmer/home.grace/eec42/BirdDetector/src/data_preprocessing


In [2]:
device = "0" if torch.cuda.is_available() else "cpu"
if device == "0":
    torch.cuda.set_device(0) # Set to your desired GPU number

print(device)

0


In [3]:
# Create a new YOLO model from scratch
#model = YOLO('yolov8n.yaml')

# Load a pretrained YOLO model (recommended for training) - for now, we keep the small version
model = YOLO('yolov8n.pt')

In [4]:
# TRAIN the model on our dataset (data.yml config file) 
model_name = 'pfeifer_yolov8n_70epoch_default_batch64'

results = model.train(
   data='data.yaml',
   #imgsz=480,  # we are trying with several img size so we do not precise the size -> will automatically resize all images to 640x640
   epochs=70,
   batch=64,
   #cos_lr=True,
   #dropout=0.3,
   #optimizer='Adam',
   patience=30,
   device=0,
   verbose=True,
   #degrees=90, fliplr=0.9, flipud=0.9, # augmentation parameters
   name=model_name)

New https://pypi.org/project/ultralytics/8.0.208 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA RTX A5000, 24248MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=70, patience=30, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=pfeifer_yolov8n_70epoch_default_batch64, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=Fa

THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_no_background/train
THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_no_background/train


train: Scanning /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_no_background/train/labels.cache... 2264 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2264/2264 [00:00<?, ?it/s]
train: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_no_background/train/images/global_birds_pfeifer_Rzepecki Islands_middle_2016_Chinstrap_penguins_115_patch_0.0_0.0_448_448.jpg: 8 duplicate labels removed
train: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_no_background/train/images/global_birds_pfeifer_Rzepecki Islands_middle_2016_Chinstrap_penguins_115_patch_0.0_2.0_448_448.jpg: 8 duplicate labels removed
train: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_no_background/train/images/global_birds_pfeifer_Rzepecki Islands_middle_2016_Chinstrap_penguins_115_patch_2.0_0.0_448_448.jpg: 8 duplicate labels removed
train: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_no_background/train/images/global_birds_pfeifer_Rzepecki Isla

THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_no_background/val
THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_no_background/val


val: Scanning /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_no_background/val/labels.cache... 358 images, 0 backgrounds, 0 corrupt: 100%|██████████| 358/358 [00:00<?, ?it/s]
val: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_no_background/val/images/global_birds_pfeifer_Rzepecki Islands_south_2016_Chinstrap_penguins_103_patch_0.0_0.0_448_448.jpg: 176 duplicate labels removed
val: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_no_background/val/images/global_birds_pfeifer_Rzepecki Islands_south_2016_Chinstrap_penguins_103_patch_0.0_2.0_448_448.jpg: 177 duplicate labels removed
val: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_no_background/val/images/global_birds_pfeifer_Rzepecki Islands_south_2016_Chinstrap_penguins_103_patch_2.0_0.0_448_448.jpg: 177 duplicate labels removed
val: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_no_background/val/images/global_birds_pfeifer_Rzepecki Islands_south_2016_Chins

## Evaluation

In [5]:
# EVALUATE the model's performance on the test set
metrics = model.val(split='test', save_json=True)

# Export the model to ONNX format
#success = model.export(format='onnx')

Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA RTX A5000, 24248MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_no_background/test/images
THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_no_background/test/images


val: Scanning /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_no_background/test/labels.cache... 720 images, 0 backgrounds, 0 corrupt: 100%|██████████| 720/720 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:07<00:00,  1.51it/s]
                   all        720      15800      0.832      0.697      0.702       0.31
Speed: 0.1ms preprocess, 1.4ms inference, 0.0ms loss, 3.4ms postprocess per image
Saving runs/detect/pfeifer_yolov8n_70epoch_default_batch642/predictions.json...
Results saved to runs/detect/pfeifer_yolov8n_70epoch_default_batch642


In [6]:
metrics.box.map    # map50-95
#metrics.box.map50  # map50
#metrics.box.map75  # map75
#metrics.box.maps   # a list contains map50-95 of each category

0.31010126131588556

In [7]:
# EVALUATE the model's performance on the test set
metrics = model.val(split='test', save_json=True, iou=0.5, max_det=600)

# Export the model to ONNX format
#success = model.export(format='onnx')

Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA RTX A5000, 24248MiB)


THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_no_background/test/images
THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_no_background/test/images


val: Scanning /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_no_background/test/labels.cache... 720 images, 0 backgrounds, 0 corrupt: 100%|██████████| 720/720 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:07<00:00,  1.55it/s]
                   all        720      15800      0.819      0.759      0.751      0.307
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 3.3ms postprocess per image
Saving runs/detect/pfeifer_yolov8n_70epoch_default_batch642/predictions.json...
Results saved to runs/detect/pfeifer_yolov8n_70epoch_default_batch642


In [8]:
metrics.box.map    # map50-95

0.3065920804095276

## Visualization

In [9]:
#model = YOLO('runs/detect/pfeifer_yolov8n_70epoch_default/weights/best.pt')
model_name = 'pfeifer_yolov8n_70epoch_default_batch64'

In [10]:
# Select randomly 10 images from the test dataset
selected_img = random.choices(os.listdir('/gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_no_background/test/images/'), k=5)

results = model.predict(
        #model = 'runs/detect/pfeifer_yolov8n_70epoch_default_batch32_dropout0.3',
        source = [os.path.join('/gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_no_background/test/images/', img) for img in selected_img],
        conf = 0.2, 
        iou = 0.5,
        show=False,
        save=False
    )


0: 640x640 56 birds, 1: 640x640 6 birds, 2: 640x640 12 birds, 3: 640x640 55 birds, 4: 640x640 12 birds, 79.7ms
Speed: 1.7ms preprocess, 15.9ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)


In [11]:
for img, result in zip(selected_img, results):

    detection_boxes = []
    save_path = '/vast/palmer/home.grace/eec42/BirdDetector/src/model/runs/detect/' + model_name + '/prediction_' + os.path.basename(result.path).split('.png')[0] + '.jpg'
    for detect in range(len(result.boxes.cls)):
        det = {}
        det['conf'] = result.boxes.conf[detect].cpu()
        det['category'] = result.boxes.cls[detect].cpu()
        coords = result.boxes.xywhn[detect].cpu()
        det['bbox'] = [coords[0]-coords[2]/2, coords[1]-coords[3]/2, coords[2], coords[3]]
        detection_boxes.append(det)
        
    im_path = os.path.join('/gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_no_background/test/images/', img)
    visutils.draw_bounding_boxes_on_file(im_path, save_path, detection_boxes,
                                    confidence_threshold=0.0, detector_label_map=None,
                                    thickness=1,expansion=0, colormap=['Red'])

    selected_label = '/gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_no_background/test/labels/' + os.path.basename(result.path).split('.jpg')[0] + '.txt'
    detection_boxes = []
    df = pd.read_csv(selected_label, sep='\t', header=None, index_col=False)
    for irow, row in df.iterrows():  
        det = {}
        det['conf'] = None
        det['category'] = row[0]
        det['bbox'] = [row[1]-row[3]/2, row[2]-row[4]/2, row[3], row[4]]
        detection_boxes.append(det)
    
    # Draw annotations
    save_path2 = '/vast/palmer/home.grace/eec42/BirdDetector/src/model/runs/detect/' + model_name + '/prediction_label_' + os.path.basename(result.path).split('.png')[0] + '.jpg'
    visutils.draw_bounding_boxes_on_file(save_path, save_path2, detection_boxes,
                                    confidence_threshold=0.0, detector_label_map=None,
                                    thickness=1,expansion=0, colormap=['SpringGreen'])
        